In [7]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from elasticsearch.client import IngestClient
import os
import json

In [8]:
client = Elasticsearch()


In [9]:
def make_documents(data, index_name):
    for event in data:
        doc = {
                '_index': index_name,
                '_type': 'document',
                '_id': event['_id'],
                'pipeline': 'testpipeline',
                '_source': {'id': event['_id'], 
                            '@timestamp': event['source']['date'], 
                            'tag': event['source']['tags'],
                            'context': event['source']['context']
                           }
        }
        yield(doc) 

In [10]:
index_name = 'testdata'

ic = IngestClient(client)
pipeline_id = 'testpipeline'
pipeline_body = {
              "description": "hourly date-time index naming",
              "processors" : [
                {
                  "date_index_name" : {
                    "field" : "@timestamp",
                    "index_name_prefix" : index_name + "-",
                    "date_rounding" : "w",
                  }
                }
              ]
}
ic.put_pipeline(id=pipeline_id, body=pipeline_body)



{'acknowledged': True}

In [11]:
ic.get_pipeline(id='testpipeline')



{'testpipeline': {'description': 'hourly date-time index naming',
  'processors': [{'date_index_name': {'date_rounding': 'w',
     'field': '@timestamp',
     'index_name_prefix': 'testdata-'}}]}}

In [12]:
destdir = './testdata'
files = [ f for f in os.listdir(destdir) if os.path.isfile(os.path.join(destdir,f)) ]
result = ''
for filename in files:
    with open(os.path.join(destdir,filename)) as f:
        data = json.load(f)
        result = bulk(client, make_documents(data, index_name))

print(result)    
#bulk(client, make_documents(data, index_name))

(746, [])
